In [24]:
-- option settings
local rootDir = '../'
local date = '1118'
-- local dateLoad = '1108'
-- local itLoadFix = ''
-- local dateLoad = '1107'
-- local itLoadFix = '_iteration_80000'
local dateLoad = '1117'
local itLoadFix = '_iteration_120000'

local align = "_nowarp"
local modelName = 'model2_symskip_nngraph2_deeper'
local epochTest = '400' -- don't know what this means

local dataFolder = rootDir.."data/training_augumented_all_nostab"..align
local modelDir = rootDir.."models/"..modelName..".lua"
local paramLoadFolder = rootDir..'logs/'..dateLoad..'_'..modelName..align
local paramSaveFolder =rootDir..'logs/'..date..'_'..modelName..align

opt = {}
opt.seed = 251
opt.model = modelDir
opt.model_param_load = paramLoadFolder..'/param_epoch_'..epochTest..itLoadFix..'.t7' 
opt.bn_meanstd_load = paramLoadFolder..'/bn_meanvar_epoch_'..epochTest..itLoadFix..'.t7'
opt.optimstate_load = paramLoadFolder..'/optimstate_epoch_'..epochTest..itLoadFix..'.t7'
opt.data_root = dataFolder
opt.trainset_size = 61 
opt.batch_size = 8 
opt.it_max = 10
opt.model_param = paramSaveFolder..'/param_epoch_'..epochTest 
opt.bn_meanstd = paramSaveFolder..'/bn_meanvar_epoch_'..epochTest 
opt.optimstate = paramSaveFolder..'/optimstate_epoch_'..epochTest  
opt.save_every = 1000 
-- opt.log = paramSaveFolder..'/train.log' 

paramsLoadDir = opt.model_param_load
bn_meanvarLoadDir = opt.bn_meanstd_load
optimstateLoadDir = opt.optimstate_load

In [2]:
-- scan batches
require("lfs")

alignFolder = opt.data_root
print(alignFolder)
print(string.format('scanning videos from %s',alignFolder))
videoNames = {}
batchDirs = {}
local iVideos = 0;
local nBatches = 0;
for videoName in lfs.dir(alignFolder) do
    if videoName ~= "." and videoName ~= ".." then
--         print(videoName)
        iVideos = iVideos+1
        videoNames[iVideos] = videoName
        batchDirs[videoName] = {};
        local videoFolder = paths.concat(alignFolder,videoName);
        for batchName in lfs.dir(videoFolder) do
            if batchName ~= "." and batchName ~= ".." then
                table.insert(batchDirs[videoName],
                    paths.concat(videoFolder,batchName))
--                 print(batchName)
            end
        end
        table.sort(batchDirs[videoName])
        nBatches = nBatches + #batchDirs[videoName]
    end
end
table.sort(videoNames)
print(string.format('Found %d videos and %d batches',#videoNames,nBatches))


../data/training_augumented_all_nostab_nowarp	
scanning videos from ../data/training_augumented_all_nostab_nowarp	


Found 71 videos and 33540 batches	


In [3]:
package.path = package.path .. ";../models/model2_symskip_nngraph2_deeper.lua"
print(package.path)


/home/omnisky/.luarocks/share/lua/5.1/?.lua;/home/omnisky/.luarocks/share/lua/5.1/?/init.lua;/home/omnisky/torch/install/share/lua/5.1/?.lua;/home/omnisky/torch/install/share/lua/5.1/?/init.lua;./?.lua;/home/omnisky/torch/install/share/luajit-2.1.0-beta1/?.lua;/usr/local/share/lua/5.1/?.lua;/usr/local/share/lua/5.1/?/init.lua;../models/model2_symskip_nngraph2_deeper.lua	


In [4]:
-- split datasets (videoNames) into trainset and validset
local trainsetSize = math.min(opt.trainset_size,#videoNames)

math.randomseed(opt.seed) 
local name2weight = {}
for iVideo,videoName in ipairs(videoNames) do
    name2weight[videoName] = math.random()
end
table.sort(videoNames,function(a,b) return name2weight[a]<name2weight[b] end)
trainsetNames={}
validsetNames={}
for iVideo,videoName in ipairs(videoNames) do
    if(iVideo<=trainsetSize) then
        table.insert(trainsetNames,videoName)
    else
        table.insert(validsetNames,videoName)
    end
end
print(string.format('Found %d trainsets and %d validsets',#trainsetNames,#validsetNames))

local function getDirsFromNames(names)
    local dirs = {}
    for iName,name in ipairs(names) do
        local dirsName = batchDirs[name]
        for iDir,dir in ipairs(dirsName) do
            table.insert(dirs,dir)
        end
    end
    return dirs
end

trainsetDirs = getDirsFromNames(trainsetNames)
validsetDirs = getDirsFromNames(validsetNames)
print(#trainsetDirs,#validsetDirs)

Found 61 trainsets and 10 validsets	


29110	4430	


In [5]:
-- load and show first batch
require 'math'
matio = require 'matio'

function loadBatch(videoNames,iVideo,iBatch)
    sampleDir = batchDirs[videoNames[iVideo] ][iBatch]
--     print('loading batch ' .. sampleDir)
    batchSample = matio.load(sampleDir)
    return batchSample.batchInputTorch, batchSample.batchGTTorch
end

batchInputSample,batchGTSample = loadBatch(trainsetNames,1,1);
print(#batchInputSample)
print(#batchGTSample)


function showSampleBatch(batchInput,batchGT)
    print('showing patchInput')
    local patchInputSample = batchInput[1]
--     itorch.image(patchInputSample:contiguous():view(5,3,128,128))
    
    print('showing patchGT')
    local patchGTSample = batchGT[1]
--     itorch.image(patchGTSample)
end
showSampleBatch(batchInputSample,batchGTSample)

In [6]:
-- load and view random batch sample
function loadRandomBatchFrom(batchDirs,batchSize)
    local iBatch = math.random(#batchDirs)
    local sampleDir = batchDirs[iBatch]
    local batchSample = matio.load(sampleDir)

    local batchInputRaw,batchGTRaw = batchSample.batchInputTorch, batchSample.batchGTTorch

    batchSize = batchSize or batchInputRaw:size(1)
    if batchSize - batchInputRaw:size(1) ~= 0 then
        local shuffle = torch.randperm(batchInputRaw:size(1))
        batchInput = torch.zeros(batchSize,batchInputRaw:size(2),batchInputRaw:size(3),batchInputRaw:size(4))
        batchGT = torch.zeros(batchSize,batchGTRaw:size(2),batchGTRaw:size(3),batchGTRaw:size(4))
        for i=1,batchSize do
            batchInput[i] = batchInputRaw[shuffle[i] ]
            batchGT[i] = batchGTRaw[shuffle[i] ]
        end
    else
        batchInput = batchInputRaw
        batchGT = batchGTRaw
    end

    return batchInput, batchGT
end

math.randomseed(251) 
batchInput,batchGT = loadRandomBatchFrom(batchDirs[videoNames[1]])
print(#batchInput)
print(#batchGT)

showSampleBatch(batchInput,batchGT)

  64
  15
 128
 128
[torch.LongStorage of size 4]

  64
   3
 128
 128
[torch.LongStorage of size 4]

showing patchInput	
showing patchGT	


  64
  15
 128
 128
[torch.LongStorage of size 4]

  64
   3
 128
 128
[torch.LongStorage of size 4]

showing patchInput	
showing patchGT	


In [7]:
-- load model
require 'torch'
require 'cunn'
require 'cutorch'
require 'nn'
require 'optim'

modelDir = opt.model
print(string.format( "Loading model %s",modelDir ))
require(modelDir)
net = create_model()
net = net:cuda()
print("Model loaded")

Loading model ../models/model2_symskip_nngraph2_deeper.lua	


Model loaded	


In [8]:
-- params setting
max_intensity = 255
batchSize = opt.batch_size
decayFrom = 24000
decayEvery = 8000
decayRate = 0.5
itMax = opt.it_max
lrMin = 1e-6

print(string.format( "Params setting: max_intensity = %f",max_intensity ))
print(string.format( "                    batchSize = %d",batchSize ))
print(string.format( "                    decayFrom = %d",decayFrom ))
print(string.format( "                   decayEvery = %d",decayEvery ))
print(string.format( "                    decayRate = %f",decayRate ))
print(string.format( "                        itMax = %d",itMax ))
print(string.format( "                        lrMin = %f",lrMin ))

criterion = nn.MSECriterion()
criterion = criterion:cuda()

params, gradParams = net:getParameters()

paramsSaveDir = opt.model_param
bn_meanvarSaveDir = opt.bn_meanstd
optimstateSaveDir = opt.optimstate
saveEvery = opt.save_every

Params setting: max_intensity = 255.000000	
                    batchSize = 8	
                    decayFrom = 24000	
                   decayEvery = 8000	
                    decayRate = 0.500000	
                        itMax = 10	
                        lrMin = 0.000001	


In [9]:
-- load params
function loadParams()
    print(string.format(
    "Loading params from: \nparamsLoadDir: %s \nbn_meanvarLoadDir: %s \noptimstateLoadDir: %s ", 
    paramsLoadDir,bn_meanvarLoadDir,optimstateLoadDir))
    if not pcall(
            function()
                local paramsSave = torch.load(paramsLoadDir)
                local bn_meanvarSave = torch.load(bn_meanvarLoadDir)
                optimstateSave = torch.load(optimstateLoadDir)

                params:copy(paramsSave)
                local bn_mean, bn_std = table.unpack(bn_meanvarSave)
                for k,v in pairs(net:findModules('nn.SpatialBatchNormalization')) do
                    v.running_mean:copy(bn_mean[k])
                    v.running_var:copy(bn_std[k])
                end
            end) 
    then
        print('No params to load or wrong dir!\n')
    end
    
    optimConfig = optimstateSave or {}
    optimConfig.learningRate = optimConfig.learningRate or 0.005
    optimConfig.weightDecay = optimConfig.weightDecay or 0
    optimConfig.beta1 = optimConfig.beta1 or 0.9
    optimConfig.beta2 = optimConfig.beta2 or 0.999
    optimConfig.epsilon = optimConfig.epsilon or 1e-8
    print('optimConfig = ')
    print(optimConfig)
    optimConfig.t = optimConfig.t or 0
    itBegin = optimConfig.t+1
end

loadParams()

Loading params from: 
paramsLoadDir: ../logs/1112_model2_symskip_nngraph2_deeper_nowarp/param_epoch_400_iteration_80000.t7 
bn_meanvarLoadDir: ../logs/1112_model2_symskip_nngraph2_deeper_nowarp/bn_meanvar_epoch_400_iteration_80000.t7 
optimstateLoadDir: ../logs/1112_model2_symskip_nngraph2_deeper_nowarp/optimstate_epoch_400_iteration_80000.t7 	


optimConfig = 	
{
  v : CudaTensor - size: 15305291
  m : CudaTensor - size: 15305291
  learningRate : 1.953125e-05
  t : 80000
  denom : CudaTensor - size: 15305291
  epsilon : 1e-08
  weightDecay : 0
  beta2 : 0.999
  beta1 : 0.9
}


In [10]:
-- save params
function saveParams(it)
    local paramsSaveDir = paramsSaveDir..'_iteration_'..it..'.t7'
    local bn_meanvarSaveDir = bn_meanvarSaveDir..'_iteration_'..it..'.t7'
    local optimstateSaveDir = optimstateSaveDir..'_iteration_'..it..'.t7'
    print(string.format(
        "Saving params to: \nparamsSaveDir: %s \nbn_meanvarSaveDir: %s \noptimstateSaveDir: %s ", 
        paramsSaveDir,bn_meanvarSaveDir,optimstateSaveDir))
    torch.save(paramsSaveDir,params)
    local bn_mean = {}
    local bn_std = {}
    for k,v in pairs(net:findModules('nn.SpatialBatchNormalization')) do
        table.insert(bn_mean,v.running_mean)
        table.insert(bn_std,v.running_var)
    end
    local bn_meanvar = {bn_mean,bn_std}
    torch.save(bn_meanvarSaveDir,bn_meanvar)
    torch.save(optimstateSaveDir,optimConfig)
    print(optimConfig)
end

In [11]:
-- iteration funtion
function feval(params)
    gradParams:zero()

    local batchInput,batchGT = loadRandomBatchFrom(trainsetDirs,batchSize)

    batchInput = batchInput:double():div(max_intensity):cuda()
    batchGT = batchGT:double():div(max_intensity):cuda()
        
    local predict = net:forward(batchInput)
    loss = criterion:forward(predict, batchGT)
    local dloss_dpredict = criterion:backward(predict,batchGT)
    local gradInput = net:backward(batchInput, dloss_dpredict)
        
    return loss, gradParams
end


-- Log results to files
if opt.log then
    print('logging to: '..opt.log)
    trainLogger = optim.Logger(opt.log)
    trainLogger:setNames{'loss'}
end

In [12]:
-- train
-- at least 12 GB video memory required for batch size 64
math.randomseed(sys.clock()) 
local tic = sys.clock()
net:training() -- set train = true
for it = itBegin,itMax do
    if it>=decayFrom and (it-decayFrom)%decayEvery==0 then
        optimConfig.learningRate = optimConfig.learningRate*decayRate
    end
    if optimConfig.learningRate<lrMin then
        optimConfig.learningRate = lrMin
    end

    optim.adam(feval,params,optimConfig)
    
    local toc = sys.clock()
    speed = 1/(toc-tic)
    avgSpeed = avgSpeed or speed
    avgSpeed = 0.1*speed+0.9*avgSpeed
    timeLeft = (itMax-it)/avgSpeed
    toc = tic
    tic = sys.clock()
    timeLeftMin = timeLeft/60
    print(string.format(
            'it: %d, loss: %f, lr: %f, spd: %.2f it/s, avgSpd: %.2f it/s, left: %d h %.1f min',
            it,loss,optimConfig.learningRate,speed,avgSpeed,timeLeftMin/60,timeLeftMin%60))
    
    -- update logger/plot
    if trainLogger then
        trainLogger:add{loss}
        trainLogger:style{'-'}
        trainLogger:plot()
    end

--     if it%opt.save_every == 0 or it == itMax then 
--         saveParams(it)
--     end
end

In [13]:
-- save params
saveParams(itMax)

cannot open <../logs/1114_model2_symskip_nngraph2_deeper_nowarp/param_epoch_400_iteration_10.t7> in mode  w at /home/omnisky/torch/pkg/torch/lib/TH/THDiskFile.c:673
stack traceback:
	[C]: at 0x7f6127cbc710
	[C]: in function 'DiskFile'
	/home/omnisky/torch/install/share/lua/5.1/torch/File.lua:385: in function 'save'
	[string "-- save params..."]:9: in function 'f'
	[string "local f = function() return -- save params..."]:2: in main chunk
	[C]: in function 'xpcall'
	/home/omnisky/torch/install/share/lua/5.1/itorch/main.lua:210: in function </home/omnisky/torch/install/share/lua/5.1/itorch/main.lua:174>
	/home/omnisky/torch/install/share/lua/5.1/lzmq/poller.lua:80: in function 'poll'
	...e/omnisky/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...e/omnisky/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...e/omnisky/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/home/omnisky/torch/install/share/lua/5.1/itorch/

Saving params to: 
paramsSaveDir: ../logs/1114_model2_symskip_nngraph2_deeper_nowarp/param_epoch_400_iteration_10.t7 
bn_meanvarSaveDir: ../logs/1114_model2_symskip_nngraph2_deeper_nowarp/bn_meanvar_epoch_400_iteration_10.t7 
optimstateSaveDir: ../logs/1114_model2_symskip_nngraph2_deeper_nowarp/optimstate_epoch_400_iteration_10.t7 	


In [25]:
-- load params
loadParams()

Loading params from: 
paramsLoadDir: ../logs/1117_model2_symskip_nngraph2_deeper_nowarp/param_epoch_400_iteration_120000.t7 
bn_meanvarLoadDir: ../logs/1117_model2_symskip_nngraph2_deeper_nowarp/bn_meanvar_epoch_400_iteration_120000.t7 
optimstateLoadDir: ../logs/1117_model2_symskip_nngraph2_deeper_nowarp/optimstate_epoch_400_iteration_120000.t7 	


optimConfig = 	
{
  v : CudaTensor - size: 15305291
  m : CudaTensor - size: 15305291
  learningRate : 4.8828125e-06
  t : 120000
  denom : CudaTensor - size: 15305291
  epsilon : 1e-08
  weightDecay : 0
  beta2 : 0.999
  beta1 : 0.9
}



In [26]:
-- test net

-- load batch
local lossTest = 0
local MaxLossTest = 0
local nBatchesDone = 0
local nBatchesTest = 50

net:evaluate() -- set train = false
math.randomseed(opt.seed) 
for iBatch = 1,nBatchesTest do
    local batchInput,batchGT = loadRandomBatchFrom(validsetDirs,batchSize)
    batchInput = batchInput:float():div(max_intensity):cuda()
    batchGT = batchGT:float():div(max_intensity):cuda()

    local predict = net:forward(batchInput)
    local loss = criterion:forward(predict, batchGT)
    MaxLossTest = math.max(MaxLossTest,loss)
    lossTest = lossTest+loss
    if iBatch%5 == 0 then
        print(string.format('%d/%d',iBatch,nBatchesTest))
    end
end
lossTest = lossTest/nBatchesTest
print(string.format('final test loss: %f, max test loss: %f',lossTest,MaxLossTest))

5/50	


10/50	


15/50	


20/50	


25/50	


30/50	


35/50	


40/50	


45/50	


50/50	
final test loss: 0.001696, max test loss: 0.007989	


In [11]:
print('showing patchInput')
batchInputSample,batchGTSample = loadBatch(trainsetNames,2,12);
patchInput = batchInputSample[{{1},{},{},{}}]
-- itorch.image(patchInput:contiguous():view(5,3,128,128))

print('showing patchGT')
-- show example output
patchGT = batchGTSample[1]:double():div(max_intensity):cuda()
-- itorch.image(patchGT)

print('showing net output')
patchInput = patchInput:double():div(max_intensity):cuda()
net:evaluate() -- set train = false
local predict = net:forward(patchInput)
local loss = criterion:forward(predict,patchGT)
print(loss)
predict = predict:mul(max_intensity):byte()
-- itorch.image(predict)


showing patchInput	


showing patchGT	
showing net output	


0.0039606485515833	


In [26]:
-- try overfiting a patch
local function fevalOverfit(params)
    gradParams:zero()
        
    local predict = net:forward(patchInput)
    loss = criterion:forward(predict, patchGT)
    local dloss_dpredict = criterion:backward(predict,patchGT)
    local gradInput = net:backward(patchInput, dloss_dpredict)
        
    return loss, gradParams
end

-- settings
local decayFrom = 500
local decayEvery = 300
local optimConfig = {}
optimConfig.learningRate =  0.005
optimConfig.weightDecay = 0
optimConfig.beta1 = 0.9
optimConfig.beta2 = 0.999
optimConfig.epsilon = 1e-8
local lrMin = 1e-6
local itMaxOverfit = 1000
local tic = sys.clock()

net:training() -- set train = true
math.randomseed(sys.clock()) 
for it = 1,itMaxOverfit do
    if it>=decayFrom and (it-decayFrom)%decayEvery==0 then
        optimConfig.learningRate = optimConfig.learningRate*decayRate
    end
    if optimConfig.learningRate<lrMin then
        optimConfig.learningRate = lrMin
    end

    optim.adam(fevalOverfit,params,optimConfig)
    
    local toc = sys.clock()
    speed = 1/(toc-tic)
    avgSpeed = avgSpeed or speed
    avgSpeed = 0.1*speed+0.9*avgSpeed
    timeLeft = (itMaxOverfit-it)/avgSpeed
    toc = tic
    tic = sys.clock()
    timeLeftMin = timeLeft/60
    print(string.format(
            'it: %d, loss: %f, lr: %f, spd: %.2f it/s, avgSpd: %.2f it/s, left: %d h %.1f min',
            it,loss,optimConfig.learningRate,speed,avgSpeed,timeLeftMin/60,timeLeftMin%60))
end

it: 1, loss: 0.425336, lr: 0.005000, spd: 65.83 it/s, avgSpd: 7.88 it/s, left: 0 h 2.1 min	


it: 2, loss: 0.155717, lr: 0.005000, spd: 47.64 it/s, avgSpd: 11.86 it/s, left: 0 h 1.4 min	


it: 3, loss: 0.053008, lr: 0.005000, spd: 43.98 it/s, avgSpd: 15.07 it/s, left: 0 h 1.1 min	


it: 4, loss: 0.046632, lr: 0.005000, spd: 43.35 it/s, avgSpd: 17.90 it/s, left: 0 h 0.9 min	


it: 5, loss: 0.036919, lr: 0.005000, spd: 43.86 it/s, avgSpd: 20.49 it/s, left: 0 h 0.8 min	


...

it: 996, loss: 0.001248, lr: 0.001250, spd: 48.65 it/s, avgSpd: 48.76 it/s, left: 0 h 0.0 min	


it: 997, loss: 0.001254, lr: 0.001250, spd: 48.83 it/s, avgSpd: 48.76 it/s, left: 0 h 0.0 min	


it: 998, loss: 0.001240, lr: 0.001250, spd: 48.61 it/s, avgSpd: 48.75 it/s, left: 0 h 0.0 min	


it: 999, loss: 0.001240, lr: 0.001250, spd: 49.11 it/s, avgSpd: 48.78 it/s, left: 0 h 0.0 min	


it: 1000, loss: 0.001245, lr: 0.001250, spd: 48.92 it/s, avgSpd: 48.80 it/s, left: 0 h 0.0 min	


In [29]:
-- try overfitting several batches
videoIndexOverfit = 1
batchNumOverfit = 1
batchDirsOverfit = {}
videoNamesOverfit = videoNames[videoIndexOverfit]

for iBatch = 1,10 do
    print(videoNameOverfit[iBatch])
end
print(string.format('Overfit %d batches from video %s',batchNumOverfit,videoNameOverfit))

-- get batchDirs for overfitting
-- for iBatch = 1,batchNumOverfit do
--     table.insert(batchDirsOverfit,batchDirs[videoNamesOverfit][iBatch])
--     print(batchDirsOverfit[#batchDirsOverfit])
-- end

-- math.randomseed(251) 
-- batchInput,batchGT = loadRandomBatchFrom(batchDirsOverfit,batchSize)
-- print(#batchInput)
-- print(#batchGT)

-- showSampleBatch(batchInput,batchGT)

-- math.randomseed(sys.clock()) 
-- loadRandomBatchFrom(videoNames,batchSize)

[string "-- try overfitting several batches..."]:6: bad argument #1 to 'sort' (table expected, got string)
stack traceback:
	[C]: in function 'sort'
	[string "-- try overfitting several batches..."]:6: in main chunk
	[C]: in function 'xpcall'
	/home/omnisky/torch/install/share/lua/5.1/itorch/main.lua:210: in function </home/omnisky/torch/install/share/lua/5.1/itorch/main.lua:174>
	/home/omnisky/torch/install/share/lua/5.1/lzmq/poller.lua:80: in function 'poll'
	...e/omnisky/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...e/omnisky/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...e/omnisky/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/home/omnisky/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x00406670: 